In [2]:
!pip install PyPDF2 python-docx sentence-transformers numpy scikit-learn

In [4]:
import PyPDF2
from docx import Document

def extract_text_from_pdf(pdf_path):
    """Trích xuất văn bản từ file PDF"""
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"Lỗi khi đọc file PDF: {e}")
    return text

def extract_text_from_docx(docx_path):
    """Trích xuất văn bản từ file DOCX"""
    text = ""
    try:
        doc = Document(docx_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    except Exception as e:
        print(f"Lỗi khi đọc file DOCX: {e}")
    return text
    
docx_text = extract_text_from_docx("02.2004.NQ.HDTP.docx")

# Kết hợp văn bản từ cả hai nguồn
combined_text = docx_text
print("Đoạn trích mẫu từ văn bản pháp luật:")
print(combined_text[:500])  # In 500 ký tự đầu tiên

Đoạn trích mẫu từ văn bản pháp luật:
 
NGHỊ QUYẾT
Hướng dẫn áp dụng pháp luật trong việc giải quyết
 các vụ án dân sự, hôn nhân và gia đình
HỘI ĐỒNG THẨM PHÁN TOÀ ÁN NHÂN DÂN TỐI CAO
Căn cứ vào Luật Tổ chức Toà án nhân dân;
Để thi hành đúng và thống nhất các quy định của pháp luật trong việc giải quyết các vụ án dân sự, hôn nhân và gia đình;
Sau khi có ý kiến thống nhất của Bộ trưởng Bộ Tư pháp và Viện trưởng Viện kiểm sát nhân dân tối cao,
QUYẾT ĐỊNH
I. VIỆC ÁP DỤNG CÁC QUY ĐỊNH CỦA PHÁP LUẬT VỀ THỜI HIỆU
1. Việc áp dụng các quy đ


In [6]:
def split_into_paragraphs(text, min_length=20):
    """Tách văn bản thành các đoạn, chỉ giữ những đoạn có độ dài >= min_length ký tự"""
    paragraphs = [p.strip() for p in text.split("\n") if len(p.strip()) >= min_length]
    return paragraphs

paragraphs = split_into_paragraphs(combined_text)
print("Số đoạn trích thu được:", len(paragraphs))

Số đoạn trích thu được: 112


In [9]:
!pip install hf_xet

   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   ------------ --------------------------- 1.3/4.2 MB 6.1 MB/s eta 0:00:01
   --------------------------- ------------ 2.9/4.2 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 4.2/4.2 MB 6.4 MB/s eta 0:00:00


In [10]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np

# Tải mô hình embeddings phù hợp cho tiếng Việt
embed_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Tạo embeddings cho tất cả các đoạn văn bản trích được
print("Đang tạo embeddings cho các đoạn trích...")
paragraph_embeddings = embed_model.encode(paragraphs, convert_to_tensor=True)
print("Xong tạo embeddings cho", len(paragraphs), "đoạn văn.")

def retrieve_information(query, threshold=0.6):
    """
    Dựa trên câu hỏi của người dùng, tìm ra đoạn văn bản có cosine similarity cao nhất.
    Nếu điểm tương đồng thấp hơn ngưỡng threshold thì trả lời không tìm thấy thông tin phù hợp.
    """
    query_embedding = embed_model.encode(query, convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_embedding, paragraph_embeddings)[0]
    best_idx = int(torch.argmax(cosine_scores))
    best_score = cosine_scores[best_idx].item()
    
    if best_score < threshold:
        return "Xin lỗi, tôi không tìm thấy thông tin phù hợp trong văn bản pháp luật."
    else:
        return paragraphs[best_idx]

# Test ví dụ:
test_query = "Quy định chuyển giao quyền sử dụng đất thừa kế như thế nào?"
result = retrieve_information(test_query)
print("\nKết quả tìm kiếm:")
print(result)

Đang tạo embeddings cho các đoạn trích...
Xong tạo embeddings cho 112 đoạn văn.

Kết quả tìm kiếm:
1. Xác định quyền sử dụng đất là di sản


In [13]:
print("\nChatbot tư vấn luật pháp đã sẵn sàng. Gõ 'exit' hoặc 'thoát' để kết thúc cuộc trò chuyện.")

while True:
    user_query = input("\nBạn: ")
    if user_query.lower() in ['exit', 'thoát']:
        print("Chatbot: Cảm ơn bạn đã sử dụng dịch vụ!")
        break
    answer = retrieve_information(user_query)
    print("Chatbot:", answer)


Chatbot tư vấn luật pháp đã sẵn sàng. Gõ 'exit' hoặc 'thoát' để kết thúc cuộc trò chuyện.



Bạn:  Ai được quyền thừa kế đất đai theo pháp luật Việt Nam?


Chatbot: - Đất không bị Nhà nước quản lý trong quá trình thực hiện chính sách đất đai của Nhà nước Việt Nam dân chủ cộng hoà, Chính phủ cách mạng lâm thời Cộng hoà miền Nam Việt Nam và Nhà nước Cộng hoà xã hội chủ nghĩa Việt Nam;



Bạn:  Nếu có người bị mất tích trong hàng thừa kế thì phải làm sao?


Chatbot: b. Trường hợp người chết để lại di sản cho các thừa kế nhưng các thừa kế không trực tiếp quản lý, sử dụng mà di sản đó đang do người khác chiếm hữu bất hợp pháp hoặc thuê, mượn, quản lý theo uỷ quyền... thì các thừa kế có quyền khởi kiện người khác đó để đòi lại di sản.



Bạn:  thoát


Chatbot: Cảm ơn bạn đã sử dụng dịch vụ!
